1. **Data Collection**:
   - Gather a diverse set of high-quality dermatology images. Ensure that these images cover a wide range of skin conditions and are representative of the types of images that your assessment system will encounter.
   - Create distorted versions of these images to simulate various quality criteria such as lighting, background, field of view, orientation, focus & depth of field, resolution, and color calibration. You can use image editing software or libraries like OpenCV to introduce these distortions.

2. **Preprocessing**:
   - Resize all images to a consistent resolution to ensure uniformity in processing.
   - Normalize pixel values to a common scale (e.g., [0, 1]) to improve model convergence.

3. **Feature Extraction**:
   - Extract relevant features from the images. For example, you can use techniques like Histogram of Oriented Gradients (HOG), Local Binary Patterns (LBP), or deep learning-based feature extraction methods.

4. **Model Building**:
   - Design and train a machine learning model to assess image quality based on the extracted features.
   - Choose an appropriate model architecture based on the complexity of the task and the amount of data available. Common choices include Support Vector Machines (SVM), Random Forests, Convolutional Neural Networks (CNNs), or transfer learning from pre-trained models like VGG, ResNet, or EfficientNet.
   - Split your dataset into training, validation, and test sets to evaluate the performance of your model accurately.

5. **Evaluation**:
   - Evaluate the performance of your model using appropriate metrics such as accuracy, precision, recall, F1-score, or area under the ROC curve (AUC).
   - Use techniques like cross-validation to ensure the robustness of your model's performance.

6. **Fine-tuning and Optimization**:
   - Fine-tune your model based on the evaluation results. Adjust hyperparameters, try different model architectures, or experiment with different feature extraction techniques to improve performance.
   - Consider techniques like data augmentation to increase the diversity of your training data and improve generalization.

7. **Challenges**:
   - **Data Quality**: Ensuring that your training data accurately represents the diversity of images encountered in teledermatology can be challenging.
   - **Subjectivity**: Image quality assessment can be subjective, and different evaluators may have varying opinions on what constitutes a "good" or "bad" image.
   - **Generalization**: Ensuring that your model can generalize well to unseen images is crucial, especially when dealing with real-world applications where image quality may vary significantly.
   - **Computational Resources**: Training deep learning models can be computationally intensive, so you may need access to sufficient computational resources.

In [1]:
import os
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils.utils_data import create_distortions_batch

In [2]:
folder_path = "images"
files = [f for f in os.listdir(folder_path) if not f.startswith('.') and '__' not in f]

with tqdm(total=len(files), desc="Images to Distort") as pbar:
    for i, f in enumerate(files):
        img_path = os.path.join(folder_path, f)
        create_distortions_batch(img_path, folder_path, batch_size=32, counter=i)
        pbar.update(1)
        if i == 1:
            break

Images to Distort:   0%|          | 0/410 [00:00<?, ?it/s]/Users/choekyelnyungmartsang/opt/anaconda3/envs/ARNIQA/lib/python3.10/site-packages/torchvision/transforms/functional.py:685: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_matrix = torch.tensor(startpoints, dtype=torch.float).view(8)
Images to Distort:   0%|          | 2/410 [00:20<1:08:48, 10.12s/it]


In [3]:
from utils.utils_data import resize_crop, center_corners_crop

folder_path = "images"
files = [f for f in os.listdir(folder_path) if not f.startswith('.') and '__' not in f]

# normal crop
test_path = os.path.join(folder_path, files[1])
transform = transforms.Compose([
    #transforms.Resize(256),                      # Resize the image to 256x256
    #transforms.CenterCrop(224),                  # Center crop the image to 224x224
    transforms.ToTensor(),                       # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])
orig_image = Image.open(test_path).convert('RGB')
image = transform(orig_image)
#image = transform(image).unsqueeze(0)

# center_corners_crop
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
img1 = Image.open(test_path).convert('RGB')
img1 = center_corners_crop(img1, crop_size=224)
img1 = [transforms.ToTensor()(crop) for crop in img1]
img1 = torch.stack(img1, dim=0)
im1g = normalize(img1)

# resize_crop
img2 = Image.open(test_path).convert('RGB')
img2 = resize_crop(img2, crop_size=224)
img2 = transforms.ToTensor()(img2)

torch.Size([3, 1080, 810])

In [3]:
def display(orig_img, crop_img, distort_img):
    _, axes = plt.subplots(1, 3, figsize=(15, 5))  # Create a figure with three subplots
    axes[0].imshow(orig_img)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    axes[1].imshow(crop_img.permute(1, 2, 0).cpu().numpy().clip(0, 1))
    axes[1].set_title('Normalized Image')
    axes[1].axis('off')
    axes[2].imshow(distort_img.permute(1, 2, 0).cpu().numpy())
    axes[2].set_title('Distorted Image')
    axes[2].axis('off')

    plt.tight_layout()  # Adjust layout to prevent overlap of subplots
    plt.show()
display(orig_image, image, img_A_orig)